
# Control del PI y PID del UNDCMotor y el UNThermal

El UNDCMotor es un minilaboratorio que permite el control de velocidad y posición de un motor DC con el fin de ilustrar los principios básicos del control autómatico.


## Configuración


### Instalación de la libreria unmotor

Descomentar y ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__



In [ ]:
import subprocess
command = ["pip", "install", "-I","--user", "git+https://github.com/nebisman/UNDCMotor.git@main#subdirectory=code/python_code"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    print(line.strip())


### Instalación de la libreria unthermal

Descomentar y ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__



In [ ]:
import subprocess
command = ["pip", "install", "-I","--user", "git+https://github.com/nebisman/UNThermal.git@main#subdirectory=code/python_code"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    print(line.strip())


### Importación de comandos de la libreria unmotor y de Matplotlib 

A continuación importamos la libreria unmotor, la libreria unthermal y otras librerias necesarias para realizar trabajo adicional con las figuras. 

In [ ]:
# importacion de graficos y funciones interactivas
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# importación de las librerias unmotor, unthermal y control
import unmotor as mot
import unthermal as ter
import control as ct


---


## Control PI de una planta de primer orden

Si una planta puede ser bien aproximada por un sistema de primer orden, se puede sintetizar directamente un controlador PI a partir del modelo.



### Control PI del motor en velocidad

Vamos a hacer un controlador para el motor para la velocidad de $300^o/s$. Para ello primero vamos a hacer un modelo "fresco" del sistema y de allí tomamos los parámetros $\alpha$ y $\tau$.


In [ ]:
my_dcmotor = mot.MotorSystemIoT(plant_number = "xxxx", broker_address = "192.168.1.100");  
G1, G2 = mot.get_models_prbs(my_dcmotor, yop = 300)

####  Funciones para el calculo y simulación del controlador en lazo cerrado


Al usar un controlador PI de dos grados de libertad para una planta de primer orden de la forma

$$G(s)=\frac{\alpha}{\tau\, s +1}$$,

se obtiene la función de transferencia de lazo cerrado dada por el siguiente sistema de segundo orden

$$T(s) = \frac{Y(s)}{R(s)}=\frac{\omega_0^2}{s^2 + 2\,\zeta\,\omega_0s + \omega_0^2}$$

las constantes $k_p$ y  $k_i$ están, respectivamente, dadas por:

$$k_p = \frac{2\,\zeta\,\omega_0\,\tau-1}{\alpha}$$

$$k_i = \frac{\omega_0^2\,\tau}{\alpha}$$

Las siguientes funciones permiten calcular el controlador PI y simular su respuesta en lazo cerrado.


In [ ]:
def compute_pi(G, w0, zita):
    """
    This function computes a 2-DOF - PI controller. See Astrom and Murray "Feedback Systems: An Introduction for Scientist and Engineers"
    Parameters:
       G: system transfer function
       w0: desired natural frequency
       zita: desired damping factor     
    """    
    # plant parameters
    alpha = G.num[0][0][0] 
    tau = G.den[0][0][0]
    
    # computation of kp and ki
    kp = (2*zita*w0*tau - 1)/alpha
    ki = w0**2*tau / alpha
    return kp, ki



def simulate_closed_loop(G, T,  w0, zita, t0, t1, r0, r1):
    """
    This function simulates and plots the expected response according to the obtained model.
    Parameters:
       G: open-loop system transfer function
       T: closed-loop function
       r0: initial reference value
       r1: final reference value
       t0: time of the initial reference value
       t1: duration of the final reference value     
    """
    #compute the control signal
    Gur = ct.minreal(T/G)
    t = np.linspace(0, t0 + t1, 200)
    r = []
    for tc in t:
        if tc <= t0:
            r.append(0)
        else:
            r.append(r1-r0)
    r = np.array(r)
    tsim, ysim  = ct.forced_response(T, t, r)
    tsim, usim  = ct.forced_response(Gur, t, r)
    fig = plt.gcf()
    ay, au = fig.get_axes()
    ay.plot(tsim, ysim + r0, color = "#ff6600ff", alpha= 0.6)
    au.plot(tsim, usim, color ="#37c8abff", alpha= 1)
    ay.text(0, r1,  'Simulated output', fontsize=12, color='#ff6600ff')    
    fig.canvas.draw()
    return 

####  Definición de la planta
Mediante el siguiente codigo definimos el UNDCMotor como una planta de primer orden de la forma $G_{v}(s)=\frac{\alpha}{\tau\,s + 1}$.

In [ ]:
# aqui definimos nuestra planta

alpha = 846.5 # ponga el valor de su modelo 
tau =  0.309 # ponga el valor de su modelo
Gmotor = ct.tf(alpha, [tau, 1])

#### Diseño, calculo y respuesta al escalón con el PI

In [ ]:
# calculo del controlador
w0 = 5 # frecuencia natural 
zita = .7 # factor de amortiguamiento
kp, ki = compute_pi(Gmotor, w0, zita)

# aqui programo el controlador PID
mot.set_pid(my_dcmotor, kp=kp, ki=ki, kd=0, beta=0, output="speed")

# aqui hago el experimento de la respuesta al escalón en lazo cerrado

r0 = 0     # valor inicial de la referencia
r1 = 300   # valor final de la referencia
t0 = 0.5   # tiempo que dura en r0 
t1 = 2     # tiempo que dura en r1

mot.step_closed(my_dcmotor, t0=t0,t1=t1, r0=r0,r1=r1);

# aqui simulo la respuesta con el modelo matemático para comparar
s = ct.TransferFunction.s
T = w0**2/ (s**2 + 2*zita*w0*s + w0**2)
simulate_closed_loop(Gmotor, T, w0, zita, t0, t1, r0, r1)



### Experimentos propuestos

+ Experimente con los valores de $\omega_0$ de 5, 10, 15.  Tenga en cuenta de cambiar el parametro `t1` para ver completo el experimento hasta el estado estacionario. Discuta con su compañero los cambios que se producen. Pruebe cada controlador manualmente con el potenciómetro y "perciba" su funcionamiento en la realidad.
+ Determine experimentalmente el mejor valor de $\omega$ antes de que el desempeño en lazo cerrado se degrade completamente. 
+ Con el valor de $\omega_0$ en $15$, cambie los valores de $\zeta$ a 0.5, 0.7 y 0.9.  Discuta los cambios que se producen. Pruebe cada controlador manualmente con el potenciómetro y "perciba" su funcionamiento en la realidad.
+ Ejecute la siguiente celda y pruebe el controlador con el potenciometro para diferentes velocidades. Intente frenar o acelerar **suavemente** la rueda.  

In [ ]:
kp, ki = compute_pi(Gmotor, 20, 0.7)
# aqui programo el controlador PID
mot.set_pid(my_dcmotor, kp=kp, ki=ki, kd=0, beta=0, output="speed")
mot.step_closed(my_dcmotor, t0=t0,t1=100, r0=0,r1=300);

#### Seguimiento de perfiles de velocidad

Mediante la siguiente función podemos hacer la prueba de respuesta a un perfil. Cada punto en el tiempo es un vertice de la curva que se interpola linealmente.

In [ ]:
timevalues = [0, 2, 4 , 7, 9]
refvalues = [0, 360, 720, 360, 360]
mot.profile_closed(my_dcmotor, timevalues, refvalues);


### Preguntas

+ Según el los experimentos realizados, ¿cómo es el error de posición y el error de velocidad del motor con el controlador PI?
+ ¿Como podriamos mejorar el seguimiento de rampas, es decir, de perfiles de velocidad?



### Control PI del sistema térmico

A continuación vamos a hacer un control PI del sistema térmico siguiendo los mismos pasos.

#### Definición de la planta como sistema IoT
A continuación definimos la planta como sistema IoT

In [ ]:
# aqui definimos el sistems termico como sistema IoT
my_thermal = ter.ThermalSystemIoT(plant_number = "XXXX", broker_address = "192.168.1.100");  

#### Definición del modelo de la planta
A continuación definimos el modelo de la planta usando los parámetros encontrados en el experimento con la PRBS que mejor nos dió en la práctica anterior

In [ ]:
# aqui definimos nuestra planta
alpha =  # poner el valor de alpha de su modelo
tau =    # poner el valor de tau de su modelo
Gtermica = ct.tf(alpha, [tau, 1])

#### Diseño, cálculo y respuesta al escalón del controlador PI
A continuación calculamos el controlador PI para nuestra planta con el modelo encontrado previamente. 

In [ ]:
# aqui calculamoss nuestro controlador
w0 = 0.3    # frecuencia natural 
zita = 0.7 # factor de amortiguamiento
kp, ki = compute_pi(Gtermica, w0, zita)

# aqui probamos y programamos nuestro controlador
r0 = 50     # valor inicial de la referencia
r1 = 60     # valor final de la referencia
t0 = 50     # tiempo que dura en r0 
t1 = 50     # tiempo que dura en r1
ter.set_pid(my_thermal, kp=kp, ki=ki, kd=0, beta=0)
ter.step_closed(my_thermal, t0=t0,t1=t1, r0=r0,r1=r1);

# Aqui simulamos con el modelo nuestro controlador
T = w0**2/ (s**2 + 2*zita*w0*s + w0**2) #esta es la función de lazo cerrado
simulate_closed_loop(Gtermica, T, w0, zita, t0, t1, r0, r1)


### Experimentos propuestos

+ Cambie la frecuencia natural para los valores $\omega_0= 0.1, 0.2, 0.3$. Ajuste el parámetro `t1` convenientemente para ver el experimento completo. Discuta los cambios que se producen y pruebe el controlador resultante con los botones. 
+ Con el valor de $\omega_0$ en $0.3$, cambie los valores de $\zeta$ a 0.25 y 0.5. Discuta los cambios que se producen y pruebe el controlador con los botones. 
+ Ejecute la siguiente celda e intente perturbar la temperatura. 

In [ ]:
kp, ki = compute_pi(Gtermica, 0.4, 0.8)
# aqui programo el controlador PID
ter.set_pid(my_thermal, kp=kp, ki=ki, kd=0, beta=0)
ter.step_closed(my_thermal, t0=0, t1=120, r0=45,r1=45);


#### Prueba de termociclado

Para muchos procesos como las pruebas PCR se usan perfiles de temperatura que deben seguirse con precisión. A continuación mostramos el seguimiento de nuestro sistema a un perfil de temperatura similar al usado en pruebas PCR. 

In [ ]:
timevalues = [0, 50, 100, 150, 200, 250]
refvalues = [90, 90, 50, 50, 70,70 ]
ter.profile_closed(my_thermal, timevalues, refvalues);


### Preguntas

+  Para mejorar la respuesta a perturbaciones, ¿se debe aumentar o disminuir la frecuencia natural del sistema de segundo orden?
+ ¿Como podria mejorarse la respuesta de este sistema en la prueba de termociclado?
+ ¿Aproximadamente hasta que frecuencia natural funciona apropiadamente este sistema en lazo cerrado?



## Control PID de una planta de segundo orden - Control de posición del UNDCMotor

Teniendo en cuenta que el ángulo es la integral de la velocidad angular, cuando hacemos control de posición del UNDCMotor, se obtiene un [modelo lineal de segundo orden](https://github.com/nebisman/UNDCMotor/blob/main/docs/modelo_matematico_unmotor.pdf), dado por la siguiente función de transferencia:

$$G(s)= \frac{\alpha}{s(\tau\, s + 1)}=\frac{b_0}{a_2\,s^2 + a_1\,s}$$

Note que, simplemente, multiplicamos la función de transferencia obtenida en velocidad por el factor de integración $1/s$ para obtener el ángulo. 

Con esta función de transferencia podemos sintetizar un controlador PID de dos grados de libertad que produzca la siguiente función de lazo cerrado:

$$T(s)= \frac{Y(s)}{R(s)}=\frac{n\,\omega_0^3} {(s^2 + 2\,\zeta \, \omega_0 \,s + \omega_0^2)(s + n\,\omega_0)}$$

La idea es tener polos dominantes de segundo orden y un polo lejano dado por el factor $n\omega_0$. Las constantes del PID se obtienen de la siguiente manera:

$$k_p = \frac{\omega_0^2\,(1+2\,\zeta\,n) - a_1}{b_0}$$

$$k_i = \frac{n\, \omega_0^3}{b_0}$$

$$k_d = \frac{\omega_0\,(n + 2\,\zeta) - a_2}{b_0}$$






### Obtención del modelo para control de posición

Para obtener el modelo apropiado para el control de posición, realizamos un experimento para obtener la función de transferencia del motor a bajas velocidades (por ejemplo a $100^o/s$)

In [ ]:
my_dcmotor = mot.MotorSystemIoT(plant_number = "xxxx", broker_address = "192.168.1.100");  
G1, G2 = mot.get_models_prbs(my_dcmotor, yop = 100)

In [ ]:
# aqui calculamos el PID para la planta
def compute_pid(G, w0, zita, nw):
    """
    This function computes a 2-DOF - PID controller. See Astrom and Murray
    Parameters:
       G: system transfer function
       w0: desired natural frequency
       zita: desired damping factor     
    """    
    # plant parameters
    b0 = G.num[0][0][0]
    a2 = G.den[0][0][1]
    a1 = G.den[0][0][2]

    
    # computation of kp, ki and kd
    kp = (w0**2 * (1 + 2*zita*n) - a1) / b0
    ki = n * w0**3 / b0
    kd = (w0 * (n + 2*zita) - a2) / b0
    return kp, ki, kd

In [ ]:
# aqui definimos nuestra planta
alpha =  # ponga el valor del experimento
tau =    # ponga el valor del experimento

# definimos la planta en posicion poniendo un integrador adicional representado por 
# el cero en el denominador Gpos= alpha/(tau*s^2 + s)
Gposicion = ct.tf(alpha, [tau, 1, 0])


In [ ]:
# calculo del controlador PID para una función de lazo cerrado de tercer orden

w0 = 5    # frecuencia natural 
zita = .7   # factor de amortiguamiento
n = 4        # el polo adicional se ubica en n*w0 
kp, ki, kd = compute_pid(Gposicion, w0, zita, n)

# aqui programo el controlador
mot.set_pid(my_dcmotor, kp=kp, ki=ki, kd=kd, beta=0, N=5,output="angle", deadzone=0)

# aqui hago el experimento de la respuesta al escalón en lazo cerrado
r0 = 0     # valor inicial de la referencia
r1 = 180   # valor final de la referencia
t0 = 0.5   # tiempo que dura en r0 
t1 = 4    # tiempo que dura en r1
mot.step_closed(my_dcmotor, t0=t0,t1=t1, r0=r0,r1=r1);

# aqui simulo la respuesta con el modelo matemático para comparar
s = ct.TransferFunction.s
T = n*w0**3/((s**2 + 2*zita*w0*s + w0**2)*(s + n*w0))
simulate_closed_loop(Gmotor, T, w0, zita, t0, t1, r0, r1)



### Experimentos propuestos

+ Varie $\omega$ en los valores de 1, 3, 4.5. Tenga en cuenta de cambiar el parametro t1 para ver completo el experimento hasta el estado estacionario. Pruebe cada controlador manualmente con el potenciómetro y "perciba" el control.
  
+ Determine experimentalmente el mejor valor de $\omega$ antes de que el desempeño en lazo cerrado se degrade.  

+ Con $\omega=4$ varíe el parámetro $\zeta$ en los valores de 0.3, 0.5 y 0.9. Pruebe cada controlador manualmente con el potenciómetro y "perciba" el control.

+ Diseñe una trayectoria y use la función profile_closed para ver la respuesta del controlador de posición.

---

# Preguntas

+ Cuando vamos a realizar control de posición, ¿Por qué es conveniente realizar un modelo de a bajas velocidades para el diseño y cálculo del controlador?

+ Si se requiere que el sistema de posición siga una trayectoria precisa mientras mantiene una velocidad precisa especificada, ¿Cómo debería modificarse el controlador?
